In [76]:
%matplotlib inline
from PIL import Image
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LogisticRegression
import os
import itertools
from tqdm import *
from tqdm import tqdm_notebook as tqdm
import random

## Create pairs of filenames and truth values from training data

In [154]:
fileNames = []
for file in os.listdir("data/train_1"):
    if file.endswith(".jpg"):
        fileNames.append(file)
# fileNamesDF = pd.DataFrame(fileNames)

In [155]:
def isPaintingFromPopularArtist(paintingRow,popularArtists):
    print(paintingRow)
    print(paintingRow[artist])
    return 0

In [169]:
#only take certain artist so that the probabilities aren't basically all false when doing the tupples
minNumPaintingsPerArtist = 300
trainInfo = pd.read_csv('data/train_info.csv')
mostPopularArtists = trainInfo['artist'].value_counts()[trainInfo['artist'].value_counts() > minNumPaintingsPerArtist]
# print(trainInfo[trainInfo['artist'].isin(mostPopularArtists.index)].shape)
fileNamesDF = trainInfo[trainInfo['artist'].isin(mostPopularArtists.index)]['filename']
# print(trainInfo[trainInfo['filename'].isin(fileNames)].shape)

In [ ]:
fileNamesDF.shape

In [170]:
fractionOfData = 0.03
trainFiles = fileNamesDF.sample(frac=fractionOfData)
trainFileNames = trainFiles.values.flatten()

In [171]:
## from itertools documentation to create tupples
def product(*args, repeat=1):
    # product('ABCD', 'xy') --> Ax Ay Bx By Cx Cy Dx Dy
    # product(range(2), repeat=3) --> 000 001 010 011 100 101 110 111
    pools = [tuple(pool) for pool in args] * repeat
    result = [[]]
    for pool in tqdm(pools):
        result = [x+[y] for x in result for y in pool]
    for prod in result:
        yield tuple(prod)
def permutations(iterable, r=None):
    pool = tuple(iterable)
    n = len(pool)
    r = n if r is None else r
    for indices in product(range(n), repeat=r):
        if len(set(indices)) == r:
            yield tuple(pool[i] for i in indices)
def combinations(iterable, r):
    pool = tuple(iterable)
    n = len(pool)
    for indices in tqdm(permutations(range(n), r)):
        if sorted(indices) == list(indices):
            yield tuple(pool[i] for i in indices)

def isSameArtist(image1,image2,trainInfo):
    artist1 = trainInfo[trainInfo['filename'] == image1]['artist']
    artist2 = trainInfo[trainInfo['filename'] == image2]['artist']
    return artist1.iloc[0] == artist2.iloc[0]


In [172]:
print("Generating Training Set for " + str(len(trainFileNames)) + " files.")
trainInputs = pd.DataFrame(list(combinations(trainFileNames,2)),columns=['image1','image2'])
trainInputs.to_csv('trainingTupplesNoTruth'+ str(fractionOfData) + '-' + str(minNumPaintingsPerArtist) +'.csv')
trainInfo = pd.read_csv('data/train_info.csv')
trainInputs['sameArtist'] = pd.Series([isSameArtist(x[0],x[1],trainInfo) for x in tqdm(trainInputs.values)])
trainInputs.to_csv('trainingTupples' + str(fractionOfData) + '-' + str(minNumPaintingsPerArtist) + '.csv')

Generating Training Set for 548 files.


HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

HBox(children=(IntProgress(value=0, max=2), HTML(value='')))

HBox(children=(IntProgress(value=0, max=149878), HTML(value='')))

Exception in thread Thread-35:
Traceback (most recent call last):
  File "/Users/adriencogny/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/adriencogny/anaconda3/lib/python3.6/site-packages/tqdm/_tqdm.py", line 148, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/adriencogny/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



KeyboardInterrupt: 

In [ ]:
PercentageTrue = trainInputs[trainInputs['sameArtist'] == True].shape[0] / trainInputs.shape[0]
PercentageFalse = 1 - PercentageTrue

In [ ]:
print(PercentageTrue)
print(PercentageFalse)

## Probabilistic Model

In [ ]:
# Look at the non-selected files
testingSet=fileNamesDF.drop(trainFiles.index).sample(frac=0.01)
testFileNames = testingSet.values.flatten()

In [ ]:
testingSet.head()
testingInputs = pd.DataFrame(list(combinations(testFileNames,2)),columns=['image1','image2'])
testingInputs.to_csv('testingSet.csv')

In [ ]:
def runProbModel(tuppleFrame,probTrue):
    returnedValues = []
    returnedPredictions = []
    for i in tqdm(range(tuppleFrame.shape[0])):
        value = random.random()
        returnedValues.append(value)
        returnedPredictions.append(value < probTrue)
#     tuppleFrame['value'] = returnedValues
#     tuppleFrame['predicted'] = returnedPredictions
    return returnedPredictions 

In [ ]:
def score(testingInputs):
    equality = testingInputs['sameArtist'] == testingInputs['predictions0']
    numEqual = equality[equality == True].shape[0]
    numTotal = equality.shape[0]
    return numEqual / numTotal

In [ ]:
testingInputs
numExperiments = 1
for i in tqdm(range(numExperiments)):
    modelPredictions = runProbModel(testingInputs,PercentageTrue)
    testingInputs['predictions' + str(i)] = modelPredictions

In [ ]:
#compute truths for dev set
trainInfo = pd.read_csv('data/train_info.csv')
testingInputs['sameArtist'] = pd.Series([isSameArtist(x[0],x[1],trainInfo) for x in tqdm(testingInputs.values)])
testingInputs.to_csv('testingResults' + str(fractionOfData) + '.csv')

In [ ]:
score(testingInputs)

## Loading an Image

In [ ]:
filePair = '10.jpg'
jpgfile = Image.open("data/train_1/" + fileName)

In [ ]:
print(jpgfile.getbands())

In [ ]:
imageDF = pd.DataFrame(list(jpgfile.getdata()),columns=['red','green','blue'])

In [ ]:
imageDF['blue'].hist()

## Generating RGB Features

In [ ]:
imgFeatures = pd.concat([imageDF['red'],imageDF['green'],imageDF['blue']])

In [ ]:
dataInfo = pd.read_csv('data/all_data_info.csv')
pixelsX = dataInfo[dataInfo['new_filename'] == fileName]['pixelsx'] # pixelsx and pixelsy could also be features
pixelsY = dataInfo[dataInfo['new_filename'] == fileName]['pixelsy'] # pixelsx and pixelsy could also be features

In [ ]:
imgFeatures = imgFeatures.append(pixelsX)
imgFeatures = imgFeatures.append(pixelsY)

## Creating and training the model

In [ ]:
model = LogisticRegression()
model

# Test

In [9]:
df = pd.read_csv('data/solution_painter.csv')

In [12]:
df.describe()

,index,sameArtist
count,2.191605e+07,2.191605e+07
mean,1.095802e+07,1.311140e-02
std,6.326618e+06,1.137519e-01
min,0.000000e+00,0.000000e+00
25%,5.479012e+06,0.000000e+00
50%,1.095802e+07,0.000000e+00
75%,1.643703e+07,0.000000e+00
max,2.191605e+07,1.000000e+00
